Nome: Maycon Douglas Pereira Alves

Matrícula: 1931133015

# Modelo de pontuação de crédito

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

 # Descrição dos dados

O conjunto de dados de Home Equity (HMEQ) contém informações de linha de base e de desempenho para 5.960 empréstimos recentes de home equity. a coluna (BAD) é uma variável binária que indica se um cliente reembolsou o empréstimo ou se está inadimplente. Esse desfecho adverso ocorreu em 1.189 casos (20%).

* BAD: 1 = cliente que está inadimplente, 0 = empréstimo reembolsado
* LOAN: Montante do pedido de empréstimo
* MORTDUE: Valor da dívida do empréstimo existente
* VALUE: Valor da propriedade atual
* REASON: Motivo do empréstimo - DebtCon = pagamentos da dívida, HomeImp = melhoria da casa
* JOB: Seis categorias ocupacionais
* YOJ: Anos no emprego atual
* DEROG: Número dos principais relatórios depreciativos
* DELINQ: Número de linhas de crédito inadimplentes
* CLAGE: Idade da linha comercial mais antiga em meses
* NINQ: Número de linhas de crédito recentes
* CLNO:  Número de linhas de crédito
* DEBTINC: juros da divída

In [ ]:
df = pd.read_csv('/kaggle/input/hmeq-data/hmeq.csv')

In [ ]:
df.info()

In [ ]:
df.sample(10)

* 0 = empréstimo reembolsado
* 1 = cliente está inadimplente 

In [ ]:
df['BAD'].value_counts()

## Tratamento para valores nulos

In [ ]:
df.isnull().sum()

Aplicando a moda no valores nulos das variáveis NINQ, DELINQ, DEROG e CLNO.

In [ ]:
def set_mode(df, var):
    df.loc[(df[var].isnull()) & (df['BAD'] == 1), var] = df.loc[(df[var].notnull()) & (df['BAD'] == 1), var].mode()[0]
    df.loc[(df[var].isnull()) & (df['BAD'] == 0), var] = df.loc[(df[var].notnull()) & (df['BAD'] == 0), var].mode()[0]


In [ ]:
set_mode(df, 'NINQ')
set_mode(df, 'DELINQ')
set_mode(df, 'DEROG')
set_mode(df, 'CLNO')

Aplicando a média para as outras variáveis com valores nulos .

In [ ]:
df.fillna(df.mean(), inplace=True)

# Análise exploratória

In [ ]:
fig, ax = plt.subplots(figsize=(12, 5))

group = df.groupby('JOB').sum()
plt.subplot(1, 2, 1)
sns.barplot(x=group.index, y=group.BAD)
plt.subplot(1, 2, 2)

sns.barplot(x=df.JOB, y=df.LOAN)

Vemos que quando a pessoa trabalha para si propria, ela precisa de mais empréstimos e normalmente tem uma inadimplente baixa.

In [ ]:
def distplot(x, titulo, df):
    summary = pd.DataFrame(df[x].describe()).T
    summary.columns = ['Quantidade', 'Média', 'Desvio Padrão', 'Minímo','25%','50%','75%', 'Máximo']
    summary = summary.T

    fig, ax = plt.subplots(figsize=(12, 5))
    plt.subplot(1, 2, 1)

    sns.distplot(df[x])

    plt.subplot(1, 2, 2)

    table = plt.table(cellText=summary.values,
              rowLabels=summary.index,
              colLabels=summary.columns,
              cellLoc = 'right', rowLoc = 'center',
              loc='right', bbox=[.5,.05,.78,.78])

    plt.axis('off')

    table.set_fontsize(22)
    table.scale(3, 3)  # may help
    plt.title(titulo)

### Valor da propriedade

In [ ]:
distplot('VALUE', 'Distribuição da variável VALUE', df)

Vemos que 25% dos pedidos de emprestimos declaram um valor de propriedade abaixo de 67 mil dolares, 25% acima de 120 mil dolares e 50% com valores entre 67 mil e 120 dolares.

É interessante criar uma categorização de valor de propriedade, para analisarmos se existe alguma relação do tipo da casa com a quitação de empréstimo. 

* Padrão Baixo menor que 60 mil
* Padrão Médio menor que 90 mil
* Padrão Alto menor que 120 mil
* Padrão Muito Alto maior que 120 mil

In [ ]:
def proriedade(x):
    if x <= 60000:
        return 'Padrão Baixo'
    if x <= 90000:
        return 'Padrão Médio'
    if x <= 120000:
        return 'Padrão Alto'
    if x > 120000:
        return 'Padrão Muito Alto'

In [ ]:
df['TIPO_PROPRIEDADE'] = df['VALUE'].transform(proriedade)

In [ ]:
fig, ax = plt.subplots(figsize=(18, 5))


plt.subplot(1, 3, 1)
count = df.groupby('TIPO_PROPRIEDADE').count()
plt.xticks(rotation=30);
plt.title("Quantidade de Empréstimos", fontsize=20)
sns.barplot(x=count.index, y=count.BAD)

plt.subplot(1, 3, 2)
plt.xticks(rotation=30);
mean = df.groupby('TIPO_PROPRIEDADE').mean()
plt.title("Média de Valores de Empréstimos", fontsize=20)
sns.barplot(x=mean.index, y=mean.LOAN)


plt.subplot(1, 3, 3)
soma = df.groupby('TIPO_PROPRIEDADE').sum()
plt.xticks(rotation=30);
plt.title("Quantidade de Inadiplentes", fontsize=20)
sns.barplot(x=soma.index, y=soma.BAD)



Vemos que:

* Quem possui uma casa de padrão muito alto, normalmente pegam muitos emprestimos, com valores muito altos e são os que ficam menos inadimplêntes. 

* Quem possui o padrão alto de casa pegam muitos emprestimos, com valores altos e possui a segunda maior taxa de inadimplência.

* Quem possui o padrão médio de casa pegam mais emprestimos, com valores um pouco maior que as pessoas de padrão de casa baixo, mas tem a maior taxa da inadimplência.

* Quem possui o padrão baixo de casa normalmente pegam menos empréstimos, com valores mais baixos, mas tem uma taxa de inadimplêntes alta. 

### Anos no emprego atual

In [ ]:
distplot('YOJ', 'Distribuição da variável YOJ - Total', df)
distplot('YOJ', 'Distribuição da variável YOJ - Maus pagadores', df[df['BAD'] == 1])
distplot('YOJ', 'Distribuição da variável YOJ - Bons pagadores', df[df['BAD'] == 0])

Vemos que a quatidade em anos no emprego atual é distribuido a esquerda, 75% dos pedidos de emprestimos são feitos por pessoas com 12 anos ou menos no emprego atual. As pessoas normalmente pedem empréstimo quando estão em média com 8.9 anos em uma empresa.

 #### Idade da linha comercial mais antiga em meses

In [ ]:
clage = df
clage['CLAGE']= clage['CLAGE'].round()

In [ ]:
distplot('CLAGE', 'Distribuição da variável CLAGE - Total', clage)
distplot('CLAGE', 'Distribuição da variável CLAGE - Maus pagadores', clage[clage['BAD'] == 1])
distplot('CLAGE', 'Distribuição da variável CLAGE - Bons pagadores', clage[clage['BAD'] == 0])

Normalmente os maus pagadores tem uma linha de crédito mais recente que os bons pagadores, eles possuem menos vinculo como banco.

Vemos que possui outliers, olhemos o boxplot abaixo:

In [ ]:
sns.boxplot(clage['CLAGE'])

Linhas de creditos maiores que 380 meses são consideradas outliers, deve-se retirar esses valores extremos.

In [ ]:
clage = clage[clage['CLAGE'] < 380]
df = df[df['CLAGE'] < 380]

In [ ]:
distplot('CLAGE', 'Distribuição da variável CLAGE - Total', clage)
distplot('CLAGE', 'Distribuição da variável CLAGE - Maus pagadores', clage[clage['BAD'] == 1])
distplot('CLAGE', 'Distribuição da variável CLAGE - Bons pagadores', clage[clage['BAD'] == 0])

Vemos que a distribuição ficou mais normalizada após a retirada dos outliers.

### Correlação entre as variaveis

In [ ]:
corr = df.corr()

fig, ax = plt.subplots(figsize=(10,8))

colors = sns.diverging_palette(200, 0, as_cmap=True)

sns.heatmap(corr, cmap=colors, annot=True, fmt=".2f")

plt.xticks(range(len(corr.columns)), corr.columns);

plt.yticks(range(len(corr.columns)), corr.columns)

plt.show()

In [ ]:
# Criando dummys para todas as colunas categóricas
df = pd.get_dummies(df, columns=['TIPO_PROPRIEDADE', 'REASON', 'JOB'])

## Previsão

#### Realizando busca dos melhores hiper parametros para o modelo XGBClassifier

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:

# XGBoost

# Importar o modelo
from xgboost import XGBClassifier

# Instanciar o modelo
xgb = XGBClassifier(n_estimators=200, n_jobs=-1, random_state=42)


In [ ]:
hyper = {
            "learning_rate": [0.05, 0.10, 0.15, 0.20, 0.25, 0.30 ] ,
            "min_child_weight": [ 1, 3 ] ,
            "gamma": [0.1, 0.2 ],
            "colsample_bytree": [0.1, 0.3 ],
        }


In [ ]:
metricas = ['accuracy', 'recall', 'f1']

In [ ]:
xgb = XGBClassifier(n_estimators=200, n_jobs=4, random_state=42)

grid = GridSearchCV(xgb, param_grid=hyper, 
                         scoring=metricas, 
                         refit='f1', 
                         return_train_score=False)

In [ ]:
# definindo colunas de entrada
feats = [c for c in df.columns if c not in ['BAD']]

In [ ]:
grid.fit(df[feats], df['BAD'])

#### Melhor estimador:

In [ ]:
grid.best_estimator_

#### E o melhor score foi:

In [ ]:
grid.best_score_

#### Realizando separação da base em teste e treino, com uma porcentagem de 20% para teste.

In [ ]:
# Separando o dataframe

# Importando o train_test_split
from sklearn.model_selection import train_test_split

# Separando treino e teste
train, test = train_test_split(df, test_size=0.20, random_state=42)

# Não vamos mais usar o dataset de validação

train.shape, test.shape

# XGBoost

# Importar o modelo
from xgboost import XGBClassifier

# Instanciar o modelo
xgb = XGBClassifier(base_score=0.5, booster=None, colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.1, gamma=0.2, gpu_id=-1,
              importance_type='gain', interaction_constraints=None,
              learning_rate=0.05, max_delta_step=0, max_depth=6,
              min_child_weight=3, monotone_constraints=None,
              n_estimators=200, n_jobs=4, num_parallel_tree=1,
              objective='binary:logistic', random_state=42, reg_alpha=0,
              reg_lambda=1, scale_pos_weight=1, subsample=1, tree_method=None,
              validate_parameters=False, verbosity=None)


In [ ]:
# Usar o cross validation
from sklearn.model_selection import cross_val_score

scores = cross_val_score(xgb, train[feats], train['BAD'], n_jobs=-1, cv=5)

# média de score é de 0.89
scores, scores.mean()

In [ ]:
# Usando o XGB para treinamento e predição
xgb.fit(train[feats], train['BAD'])

# Fazendo predições
preds = xgb.predict(test[feats])

In [ ]:
test['BAD'].value_counts()

In [ ]:
plt.title("Tabela cruzada entre os valores preditos e os valores reais", fontsize=20)

ax = sns.heatmap(pd.crosstab(test['BAD'], preds),  annot=True, fmt="d", cmap="YlGnBu")
plt.xlabel('Predito')
plt.ylabel('Original')
ax.set_xticklabels(['BOM','MAU' ])
ax.set_yticklabels(['BOM','MAU' ])

f1 score: É um número entre 0 e 1 e é a média harmônica de recall e acurracy.


Obtemos um f1 score de 0.7167919799498746, por esse valor eu considero que temos um bom modelo.

In [ ]:
# Medir o desempenho do modelo
from sklearn.metrics import f1_score

f1_score(test['BAD'], preds)